# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey

#configure gmaps
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
df_weather = pd.read_csv('../WeatherPy/output_data/cities.csv')
# df_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Kapaa,22.08,-159.32,82.99,74,20,11.41,US,1602815670
1,Örnsköldsvik,63.29,18.72,35.60,100,1,8.05,SE,1602815886
2,Namatanai,-3.67,152.43,85.26,67,77,6.53,PG,1602815886
3,Mar del Plata,-38.00,-57.56,55.99,100,90,6.93,AR,1602815703
4,East London,-33.02,27.91,63.61,78,11,8.99,ZA,1602815668


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
#Create locations for latitude and longitdue
lat_lng = df_weather[['Lat', 'Lng']]
#lat_lng.head()

#Fill NaN values and convert to float
humidity = df_weather["Humidity"]

In [31]:
# PLot Heatmap
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))

#Create a heat layer
heat_layer = gmaps.heatmap_layer(lat_lng, weights=humidity,
                                dissipating=False, max_intensity=100, 
                                point_radius=1.5)

#Add a Heat Layer to the google maps first layer
fig.add_layer(heat_layer)

#Display Heat Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
#Find cities with:
# Max temperature lower than 80 but higher than 70 degrees
# Windspeed less than 10 mph
# Zero % cloudiness
# Create a dataframe of the best cities

df_best_cities = df_weather.loc[(df_weather["Max Temp"] <= 80) & (df_weather["Max Temp"] >= 70) & \
                (df_weather["Wind Speed"] <= 10) & (df_weather["Clouds"]== 0)].dropna()


df_best_cities.head()

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
40,Oddur,4.12,43.89,71.01,80,0,6.29,SO,1602815894
127,Januária,-15.48,-44.37,78.96,33,0,2.06,BR,1602815911
138,Hurghada,27.26,33.81,78.80,44,0,9.17,EG,1602815913
139,Piaçabuçu,-10.41,-36.43,74.14,84,0,8.46,BR,1602815901
168,Grand Gaube,-20.01,57.66,71.01,80,0,3.00,MU,1602815679


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Store into variable "hotel_df"
hotel_df = df_best_cities.loc[:, ["City", "Country", "Lat", "Lng"]]

#Add a "Hotel Name" column to the hotel dataframe
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
40,Oddur,SO,4.12,43.89,
127,Januária,BR,-15.48,-44.37,
138,Hurghada,EG,27.26,33.81,
139,Piaçabuçu,BR,-10.41,-36.43,
168,Grand Gaube,MU,-20.01,57.66,
170,Sharjah city,AE,25.36,55.40,
300,Mwense,ZM,-10.38,28.70,
316,Paragominas,BR,-2.97,-47.48,
335,Amapá,BR,1.00,-52.00,
351,Bolgatanga,GH,10.79,-0.85,


In [19]:
#Parameters to seach for hotels with 5000 meters of the best cities above
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type": "hotel",
         "keyword": "hotel", 
         "radius" : 5000, 
         "key" : gkey}

In [23]:
# Index an use "iterrows" to match the lat and lng with cities
for index, row in hotel_df.iterrows(): 
    lat = row["Lat"]
    lng = row["Lng"]
    city_names = row["City"]
    
    #Set Params
    params["location"] = f"{lat},{lng}"
    
    #Make the request to the API and convert to JSON
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save Hotel Names into a dataframe. Try and Except through the hotels 5000 mile radius of the best cities
    try: 
        print(f"Closest hotel in {city_names} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("No hotels found.")
        print("-------------------")
        
    print("Search Complete.")
    
    

Closest hotel in Oddur is Hotel Kontome 2.
Search Complete.
Closest hotel in Januária is Hotel e Restaurante Viva Maria.
Search Complete.
Closest hotel in Hurghada is Marina Star Hotel.
Search Complete.
Closest hotel in Piaçabuçu is Pousada Raizes.
Search Complete.
Closest hotel in Grand Gaube is LUX Grand Gaube Resort & Villas.
Search Complete.
Closest hotel in Sharjah city is TIME Express Hotel.
Search Complete.
No hotels found.
-------------------
Search Complete.
No hotels found.
-------------------
Search Complete.
No hotels found.
-------------------
Search Complete.
Closest hotel in Bolgatanga is GANASS HOTEL.
Search Complete.
Closest hotel in Bat Khela is Trai Hotel.
Search Complete.
Closest hotel in Rafaela is Amérian Rafaela Hotel.
Search Complete.
Closest hotel in Sayhāt is golden garden al corniche hotel.
Search Complete.


In [28]:
#Display the new hotel dataframe with the hotel names inside of the columns
hotel_df

,City,Country,Lat,Lng,Hotel Name
40,Oddur,SO,4.12,43.89,Hotel Kontome 2
127,Januária,BR,-15.48,-44.37,Hotel e Restaurante Viva Maria
138,Hurghada,EG,27.26,33.81,Marina Star Hotel
139,Piaçabuçu,BR,-10.41,-36.43,Pousada Raizes
168,Grand Gaube,MU,-20.01,57.66,LUX Grand Gaube Resort & Villas
170,Sharjah city,AE,25.36,55.40,TIME Express Hotel
300,Mwense,ZM,-10.38,28.70,
316,Paragominas,BR,-2.97,-47.48,
335,Amapá,BR,1.00,-52.00,
351,Bolgatanga,GH,10.79,-0.85,GANASS HOTEL


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…